## 104人力銀行網站爬蟲

In [1]:
import requests
from bs4 import BeautifulSoup
import time

# 設定模擬的 headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.104.com.tw/company/uboh3w8?job=&roleJobCat=0_0&area=0&page=1&pageSize=20&order=8&asc=0&jobsource=checkc&tab=job',
    'Connection': 'keep-alive'
}

# 要爬取的職位列表頁面 URL
url = 'https://www.104.com.tw/jobs/search/api/jobs?indcat=1012001000&isJobList=1&jobcat=2007000000&jobsource=m_joblist_search&page=1&pagesize=20&searchJobs=1' #更改pagesize 可以獲得更多職業



# 發送請求
response = requests.get(url, headers=headers)

time.sleep(3)

# 檢查請求是否成功
if response.status_code == 200:
    # 使用 BeautifulSoup 解析頁面
    data = response.json()
    counter =0
    for job in data["data"]:
        print(job["jobName"])
        print(job["custName"])
        print (counter)
        counter+=1
    print(data)
    

else:
    print(f"Failed to retrieve data from {url}, status code: {response.status_code}")


資深工程師
柏仁醫院
0
資訊人員
馨蕙馨醫院
1
App工程師
眾匯智能健康股份有限公司
2
【資通安全中心】資安人員
衛生福利部雙和醫院(委託臺北醫學大學興建經營)
3
【資訊室】程式開發人員
衛生福利部雙和醫院(委託臺北醫學大學興建經營)
4
【IT】MIS 工程師
科林儀器股份有限公司
5
總院 資訊部 助理程式師(約聘)
國泰醫療財團法人國泰綜合醫院
6
資訊助理
佳醫創照健康股份有限公司
7
資訊儲備主管
霧峰澄清醫院
8
總院 資訊部  程式師(網路工程師)
國泰醫療財團法人國泰綜合醫院
9
櫃臺行政專員（總薪資32000~45000 )， 打字快 50字/分 以上 熟悉電腦相關操作 資料處理 會計 相關科系佳 院內有晉升管道 可以轉美容諮詢
靚優健康醫學美容診所
10
資訊專員
佳醫創照健康股份有限公司
11
行政專員（總薪資32000~45000 )， 打字快 50字/分 以上 熟悉電腦相關操作 資料處理 會計 相關科系佳 院內有晉升管道 可以轉美容諮詢
靚優健康醫學美容診所
12
【國泰集團】系統分析師
杏保醫網股份有限公司
13
【國泰集團】MIS資訊專員 ★福利優/近捷運六張犁站★
杏保醫網股份有限公司
14
院聘資訊技師(MIS組)(本院現職人員不得報考)-資訊室派駐人事室
國立成功大學醫學院附設醫院_資訊室
15
wordpress網站架設維護工程師
昊德深層經絡按摩推拿館
16
總院 資訊部 系統網路組 助理程式師(兼職)
國泰醫療財團法人國泰綜合醫院
17
衛生福利部苗栗醫院 資訊室誠徵約用中級專員
衛生福利部苗栗醫院
18
資訊維護管理人員 (資訊室正職)-依學、經歷敘薪
國立陽明交通大學附設醫院
19
{'data': [{'appearDate': '20241226', 'applyCnt': 8, 'coIndustry': 1012001001, 'coIndustryDesc': '醫院', 'custName': '柏仁醫院', 'custNo': '14930809000', 'description': '[職務要求] \n1. 協助企業資訊發展策略執行\n    ※ 具備系統整合、規劃、開發及效能優化之經驗及能力.\n    ※ 整合內部需求，與廠家應對技術方案並督促協力廠商執行進度及技術品質. \n    ※ 

In [7]:
print(data['data']['page'])
print(data['data']['page'])
print(data['data']['page'])

TypeError: list indices must be integers or slices, not str

In [ ]:
https://www.104.com.tw/job/ajax/content/26906

https://www.104.com.tw/company/ajax/joblist/uboh3w8?roleJobCat=0_0&area=0&page=2&pageSize=40

In [8]:
import requests
from bs4 import BeautifulSoup
import time
import re

# 設定模擬的 headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.104.com.tw',
    'Connection': 'keep-alive'
}

# API URL
api_url = 'https://www.104.com.tw/jobs/search/api/jobs?indcat=1012001000&isJobList=1&jobcat=2007000000&jobsource=m_joblist_search&page=1&pagesize=10&searchJobs=1'

# 發送 API 請求
response = requests.get(api_url, headers=headers)


if response.status_code == 200:
    data = response.json()
    print(data)
    counter = 0
    for job in data["data"]:
        job_name = job["jobName"]
        cust_name = job["custName"]
        job_link = job["link"]["job"]

        print(f"{counter}. Job Name: {job_name}, Company: {cust_name}, URL: {job_link}")
        
        # 嘗試抓取薪資資訊
        try:
            job_response = requests.get(job_link, headers=headers)
            if job_response.status_code == 200:
                job_soup = BeautifulSoup(job_response.text, 'lxml')

                # 使用正則表達式搜尋薪資資訊
                salary_tag = job_soup.find(text=re.compile(r'月薪\s*\d+,\d+~\d+,\d+\s*元')) 
                if salary_tag:
                    salary_info = salary_tag.strip()  # 去除多餘空白
                    print(f"   Salary: {salary_info}")
                else:
                    # 若找不到明確的薪資範圍，則顯示其他資訊
                    salary_tag = job_soup.find('div', class_='job-description__salary')
                    if salary_tag:
                        salary_info = salary_tag.get_text(strip=True)
                        print(f"   Salary: {salary_info}")
                    else:
                        print("   Salary: Not found")
        except Exception as e:  # 加入 except 區塊處理異常
            print(f"   Error fetching salary info: {e}")

        counter += 1
        # 避免被封鎖，延遲請求
        time.sleep(1)

else:
    print(f"Failed to retrieve data from {api_url}, status code: {response.status_code}")


{'data': [{'appearDate': '20241226', 'applyCnt': 8, 'coIndustry': 1012001001, 'coIndustryDesc': '醫院', 'custName': '柏仁醫院', 'custNo': '14930809000', 'description': '[職務要求] \n1. 協助企業資訊發展策略執行\n    ※ 具備系統整合、規劃、開發及效能優化之經驗及能力.\n    ※ 整合內部需求，與廠家應對技術方案並督促協力廠商執行進度及技術品質. \n    ※ 數位電子化資訊架構之規劃設計、整合及導入,\n    ※ 跨部門協調溝通，內部流程整合及優化\n2. 機房建置整合及維運\n    ※ 熟悉Cisco Switch、Paloalto Firewall網路設備之設定與維護. \n    ※ 監控設備設定、影像下載及相關維護. \n3. 水平軸網點維運\n    ※ 電腦設備網路\n    ※ 電話線路\n    ※ 相關投放設備\n    ※ 監控設備\n    ※ 門禁\n[工作態度] \n1. 俱備良好溝通協調能力. \n2. 企業文化融入. \n[其他條件] \n1. 熟悉各種路由器、交換機、防火牆的配置、網路資安防禦及綜合佈線經驗尤佳.\n2. 工作與資訊背景相關之工作三年以上，曾擔任企業內部資訊人員尤佳.\n3. 資訊設備問題排除(協同系統商或硬體廠商). \n4.總務事務支援。\n\n', 'descSnippet': '[職務要求] \n1. 協助企業資訊發展策略執行\n    ※ 具備系統整合、規劃、開發及效能優化之經驗及能力.\n    ※ 整合內部需求，與廠家應對技術方案並督促協力廠商執行進度及技術品質. \n    ※ 數位電子化資訊架構之規劃設計、整合及導入,\n    ※ 跨部門協調溝通，內部流程整合及優化\n2. 機房建置整合及維運\n    ※ 熟悉Cisco Switch、Paloalto Firewall網路設備之設定與維護. \n    ※ 監控設備設定、影像下載及相關維護. \n3. 水平軸網點維運\n    ※ 電腦設備網路\n    ※ 電話線路\n    ※ 相關投放設備\n    ※ 監控設備\n    ※ 門禁\n[工作態度] \n1. 俱備

/tmp/ipykernel_48591/4128485037.py:40: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  salary_tag = job_soup.find(text=re.compile(r'月薪\s*\d+,\d+~\d+,\d+\s*元'))


   Salary: Not found
1. Job Name: 資訊人員, Company: 馨蕙馨醫院, URL: https://www.104.com.tw/job/8inep
   Salary: Not found
2. Job Name: App工程師, Company: 眾匯智能健康股份有限公司, URL: https://www.104.com.tw/job/6glp4
   Salary: Not found
3. Job Name: 【資通安全中心】資安人員, Company: 衛生福利部雙和醫院(委託臺北醫學大學興建經營), URL: https://www.104.com.tw/job/8bcbc
   Salary: Not found
4. Job Name: 【資訊室】程式開發人員, Company: 衛生福利部雙和醫院(委託臺北醫學大學興建經營), URL: https://www.104.com.tw/job/6zy6n
   Salary: Not found
5. Job Name: 【IT】MIS 工程師, Company: 科林儀器股份有限公司, URL: https://www.104.com.tw/job/8f1jx
   Salary: Not found
6. Job Name: 總院 資訊部 助理程式師(約聘), Company: 國泰醫療財團法人國泰綜合醫院, URL: https://www.104.com.tw/job/7qhzv
   Salary: Not found
7. Job Name: 資訊助理, Company: 佳醫創照健康股份有限公司, URL: https://www.104.com.tw/job/87hyp
   Salary: Not found
8. Job Name: 資訊儲備主管, Company: 霧峰澄清醫院, URL: https://www.104.com.tw/job/8b923
   Salary: Not found
9. Job Name: 總院 資訊部  程式師(網路工程師), Company: 國泰醫療財團法人國泰綜合醫院, URL: https://www.104.com.tw/job/6cbqb
   Salary: Not found


In [25]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json
import re  # 用於提取數字

# 初始化 webdriver
driver = webdriver.Chrome()
driver.get("https://www.104.com.tw/job/6cbqb")

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 滾動到頁面底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 等待內容加載

    # 獲取網頁 HTML 並使用 BeautifulSoup 解析
    soup = BeautifulSoup(driver.page_source, 'lxml')

    # 獲取所有 <script> 標籤
    script_tags = soup.find_all('script')

    for script_tag in script_tags:
        if script_tag.get('type') == 'application/ld+json':
            try:
                data = json.loads(script_tag.string)
                # 確認資料包含工作職位資訊
                if isinstance(data, list):
                    for item in data:
                        if item.get('@type') == 'JobPosting' and 'baseSalary' in item:
                            salary_info = item['baseSalary']
                            print(salary_info)
                            value = salary_info['value']
                            
                            # 情況 1: minValue 和 maxValue
                            if isinstance(value, dict):
                                min_salary = value.get('minValue')
                                max_salary = value.get('maxValue')
                                if min_salary and max_salary:
                                    print(f"月薪範圍: {min_salary} ~ {max_salary} {salary_info.get('currency', 'TWD')}")
                                elif value.get('value'):  # 情況 2: 單一描述或數值
                                    salary_str = value['value']
                                    match = re.search(r'(\d{1,3}(?:,\d{3})*|\d+)', salary_str)
                                    if match:
                                        extracted_salary = match.group(1).replace(",", "")
                                        print(f"月薪: {extracted_salary} {salary_info.get('currency', 'TWD')} (描述: {salary_str})")
                                else:
                                    print("未知格式，請檢查 baseSalary 資料。")
                            elif isinstance(value, str):  # 情況 2 & 3: 單一字串
                                salary_str = value
                                match = re.search(r'(\d{1,3}(?:,\d{3})*|\d+)', salary_str)
                                if match:
                                    extracted_salary = match.group(1).replace(",", "")
                                    print(f"月薪: {extracted_salary} {salary_info.get('currency', 'TWD')} (描述: {salary_str})")
                                    print()
                            else:
                                print("未知格式，請檢查 baseSalary 資料。")
            except json.JSONDecodeError:
                print("JSON 解析錯誤，跳過該標籤")

    # 計算新的頁面高度
    new_height = driver.execute_script("return document.body.scrollHeight")

    # 如果新高度與最後一個高度相同，則停止滾動
    if new_height == last_height:
        break

    last_height = new_height

driver.quit()


{'@type': 'MonetaryAmount', 'currency': 'TWD', 'value': {'@type': 'QuantitativeValue', 'unitText': 'MONTH', 'value': '月薪45,600元以上'}}
月薪: 45600 TWD (描述: 月薪45,600元以上)
